In [3]:
import pyabc
import scipy
import numpy as np
from pyabc.visualization import plot_kde_matrix_highlevel,plot_kde_matrix2,plot_kde_matrix_for_paper
from datetime import datetime
import pandas as pd
from arviz import hdi
from pyabc.visualization import plot_kde_matrix_highlevel, plot_kde_matrix
import numpy as np
import matplotlib.pyplot as plt
path = '/Users/adsurbum/sim-data/single-model-abc/new4'

f_no_aneuploidy = '2021-12-29-no-aneuploidy'
f_with_aneuploidy = '2021-12-29-an200-k1'

history = pyabc.History("sqlite:///{0}/{1}/{1}.db".format(path,f_no_aneuploidy))
history.get_all_populations()[-3:]

,t,population_end_time,samples,epsilon,particles
30,29,2022-01-05 22:32:43.491437,8276,0.277612,14
31,30,2022-01-06 20:40:28.751281,9066,0.275595,14
32,31,2022-01-08 21:03:27.959201,26766,0.272928,11


In [4]:
history = pyabc.History("sqlite:///{0}/{1}/{1}.db".format(path,f_with_aneuploidy))
history.get_all_populations()[-3:]

,t,population_end_time,samples,epsilon,particles
4,3,2021-12-29 16:37:20.673694,16125,0.145762,1111
5,4,2021-12-29 21:50:47.240188,27629,0.131083,964
6,5,2021-12-30 08:00:52.829615,53803,0.126588,982


In [9]:
def roundd(v):
    if v<1e-5:
        return round(v,8)
    elif v<1e-4:
        return round(v,7)
    else:
        return round(v,3)
    
def findd(n):
    for i in range(0,12):
        m = n*10**i
        if m>=1 and m<=10:
            return i
    raise ValueError(n)
    
def mediansplusminus(samples,kde):
    MAP = scipy.optimize.minimize(lambda x: -kde.logpdf(x) ,samples.median().values)['x']
    hi = hdi(samples.values, hdi_prob=.50).T[1]
    lo = hdi(samples.values, hdi_prob=.50).T[0]
    hi = hi*(10**np.array(list(map(findd,MAP))))
    lo = lo*(10**np.array(list(map(findd,MAP))))
    m = MAP*(10**np.array(list(map(findd,MAP))))
    plus = (hi-m)
    minus = (m-lo)
    return (m,plus,minus,list(map(findd,MAP)))

def print_mode_and_hdi(f, display_cols):
    history = pyabc.History("sqlite:///{0}/{1}/{1}.db".format(path, f))
    
    samples, weights = history.get_distribution(m=0, t=None)
    columns = list(samples.columns)
    kde = scipy.stats.gaussian_kde(samples.T.values.tolist(), weights=weights)
    size = 50000
    samples = kde.resample(size).T
    samples = pd.DataFrame(data=samples,columns=display_cols)

    ans = mediansplusminus(samples, kde)
    ans = [list(map(roundd,a)) for a in ans]
    for a in list(zip(list(samples.columns.values),*ans)):
        if a[-1]==0:
            print('${}={}\substack{{+{} \\\ -{}}}$,'.format(*a))
        else:
            print('${}={}\substack{{+{} \\\ -{}}}\\times10^{{-{}}}$,'.format(*a))
            
import warnings
warnings.filterwarnings('ignore')

In [10]:
print_mode_and_hdi(f_no_aneuploidy, display_cols = ['\mu', 'w_{2n^*}'])

$\mu=7.837\substack{+0.078 \\ -0.121}\times10^{-9}$,
$w_{2n^*}=1.013\substack{+0.0 \\ -0.0}$,


In [38]:
print_mode_and_hdi(f_with_aneuploidy, display_cols = ['\mu','\delta','w_{2n+1}','w_{2n+1^*}', 'w_{2n^*}'])

$\mu=2.93\substack{+1.122 \\ -2.824}\times10^{-6}$,
$\delta=1.716\substack{+0.974 \\ -0.384}\times10^{-3}$,
$w_{2n+1}=1.022\substack{+0.001 \\ -0.002}$,
$w_{2n+1^*}=1.025\substack{+0.001 \\ -0.001}$,
$w_{2n^*}=1.028\substack{+0.001 \\ -0.001}$,


In [ ]:
#TODO for tau=100